In [ ]:
from ilqr import iLQR
import gym
import numpy as np
import pandas as pd
import time

from aprl.agents import MujocoFiniteDiffDynamicsPerformance
from aprl.envs.mujoco_costs import get_cost
from experiments.common import set_seeds, make_env, fit_ilqr, \
                               on_iteration, receding, multi_evaluate

In [7]:
seed = 21
set_seeds(seed)

env_name = 'HalfCheetah-v2'
env, us_init = make_env(env_name, seed=seed, horizon=15)
dynamics = {'performance': MujocoFiniteDiffDynamicsPerformance(env)}
x0s = {k: dyn.get_state() for k, dyn in dynamics.items()}

costs = {k: get_cost(env_name) for k in dynamics.keys()}
ilqrs = {k: iLQR(dyn, costs[k], len(us_init))  
         for k, dyn in dynamics.items()}

In [8]:
xs, us = fit_ilqr(ilqrs, x0s, us_init, n_iterations=500)

dict_keys(['performance'])
*** Fitting performance ***
iteration 0 accepted 5.32050155928234 [-0.0444861  -0.04614066  0.08615144  0.16462926 -0.0816303   0.28175957
 -0.06175892  0.09265977  0.02311627 -0.02788938 -0.75328762 -1.36515183
  5.05002015 -1.29402765  0.96093051  1.46134204 -0.06502468 -4.19609639] [-0.06064462 -0.20915937  0.66909724  0.0730491   0.69259655 -0.3964556 ]
iteration 1 accepted 5.215307400757644 [-0.0444204  -0.05550901  0.05749703  0.17936403  0.17325539 -0.11819098
 -0.03014637  0.12040538  0.02383555 -0.0515797  -0.59562263 -1.34062591
  5.46265958 -2.38454027  2.67225654  1.5663937   0.0196618  -4.18171901] [ 0.20845221  0.96653944 -0.98207825 -0.2501812   0.5747209  -0.4790889 ]
iteration 2 accepted 4.971421145300444 [-0.04037154 -0.0621475   0.07505537  0.02944253  0.15076091  0.11592903
  0.11093913  0.12833932  0.03588408  0.13450722 -1.09240189 -0.26201127
  6.20976232 -2.67854764  1.17990689 -3.50878282  4.16328811 -3.9774914 ] [-0.76542646  0.57529

# Receding horizon

In [9]:
for k, ilqr in ilqrs.items():
    underlying = 'receding_' + k
    dynamics[underlying] = dynamics[k]
    x0s[underlying] = x0s[k]
    xs[underlying], us[underlying] = receding(ilqr, x0s[k], us_init, seed=seed, step_size=1, horizon=1000)

iteration 0 x = [[-0.01698329  0.03921564  0.09822581  0.07776713 -0.04003785  0.07786907
   0.07793559  0.09254589  0.07303366 -0.15071233  0.14185551 -0.04695025
   0.03446943  0.0666336   0.04088716  0.03092521  0.00715175  0.06794933]
 [-0.01958767  0.03856441  0.09584489  0.06603049 -0.01058884  0.06658361
   0.07767161  0.08441925  0.07385721 -0.2604382  -0.06512349 -0.2380926
  -1.17366398  2.94490085 -1.12854536 -0.02639866 -0.81266338  0.08235487]], u = [[-0.05446279  0.5580897  -0.11404657 -0.00647622 -0.13416459  0.08275776]]
iteration 1 x = [[-0.01958767  0.03856441  0.09584489  0.06603049 -0.01058884  0.06658361
   0.07767161  0.08441925  0.07385721 -0.2604382  -0.06512349 -0.2380926
  -1.17366398  2.94490085 -1.12854536 -0.02639866 -0.81266338  0.08235487]
 [-0.02274389  0.03534849  0.09022256  0.04306464  0.05408506  0.04944479
   0.08418503  0.06808364  0.07418055 -0.31562222 -0.32159211 -0.56223329
  -2.29658516  6.46738953 -1.71388277  0.65134201 -1.6335613   0.032334

iteration 21 x = [[-0.00975623 -0.13953554  0.17299526  0.1357003  -0.12351811  0.17093486
   0.50178078  0.28559644  0.27221585 -0.34069303 -0.62570812  1.18480403
  -4.85922683  9.29048343 -2.14399958  1.88523166 -4.42030913  8.68121399]
 [-0.01077191 -0.14498817  0.18564346  0.11493515 -0.07545671  0.15606452
   0.5184667   0.26138891  0.35843201 -0.10156781 -0.54526301  1.26481917
  -2.07651523  4.80614029 -1.48703334  1.66859195 -2.42075287  8.62161613]], u = [[ 0.64169874 -0.65470413  0.40835753  0.79314809  0.93635622  0.96950047]]
iteration 22 x = [[-0.01077191 -0.14498817  0.18564346  0.11493515 -0.07545671  0.15606452
   0.5184667   0.26138891  0.35843201 -0.10156781 -0.54526301  1.26481917
  -2.07651523  4.80614029 -1.48703334  1.66859195 -2.42075287  8.62161613]
 [-0.01134909 -0.15003299  0.19658063  0.10720544 -0.03633859  0.16715515
   0.50968439  0.26593884  0.43570388 -0.05771778 -0.50448232  1.09371798
  -0.77297138  3.91181234  1.10906303 -0.87823101  0.45499296  7.72

iteration 36 x = [[-5.21552662e-03 -1.53023532e-01  3.33408503e-01 -3.87596251e-02
  -1.23395418e-01  1.16518537e-01  5.52861264e-01  2.71816432e-01
   2.17402392e-01 -2.31039529e-01 -2.61685377e-01  1.23913173e+00
  -4.89592454e+00  2.90672944e-01 -6.17020022e+00  1.65456536e+00
  -2.05640350e+00 -4.34609997e+00]
 [-6.65349308e-03 -1.56953449e-01  3.42495871e-01 -8.60425845e-02
  -1.09053495e-01  7.21425398e-02  5.79596499e-01  2.56852051e-01
   1.71422661e-01 -1.43796645e-01 -3.92991739e-01  9.08736775e-01
  -4.72829594e+00  1.43419231e+00 -4.43759969e+00  2.67352341e+00
  -1.49643815e+00 -4.59797311e+00]], u = [[-0.13817676  0.13336063  0.37011499  1.07661174  0.61561618  0.02516042]]
iteration 37 x = [[-0.00665349 -0.15695345  0.34249587 -0.08604258 -0.10905349  0.07214254
   0.5795965   0.25685205  0.17142266 -0.14379665 -0.39299174  0.90873678
  -4.72829594  1.43419231 -4.43759969  2.67352341 -1.49643815 -4.59797311]
 [-0.00810626 -0.1614198   0.34854108 -0.13345319 -0.08101899  

iteration 52 x = [[ 0.05376043 -0.20350928  0.34546552  0.08562437 -0.07270775 -0.22469032
   0.45657882  0.61954965  0.30134253  0.63390923 -0.134171   -0.58133259
   3.62739823  0.41205503 -3.7681796  -4.28574524  7.26233754  3.85435246]
 [ 0.05935085 -0.20518732  0.34145891  0.11402071 -0.06577448 -0.25882906
   0.42773152  0.6724809   0.31936943  0.55904171 -0.16780382 -0.4006612
   2.83963425  0.69332664 -3.41387424 -2.8847297   5.29312491  1.80268944]], u = [[ 0.13152865 -0.04011859 -0.59544868  0.02699877  0.27833328 -0.91085487]]
iteration 53 x = [[ 5.93508451e-02 -2.05187320e-01  3.41458906e-01  1.14020708e-01
  -6.57744790e-02 -2.58829060e-01  4.27731524e-01  6.72480900e-01
   3.19369426e-01  5.59041710e-01 -1.67803821e-01 -4.00661201e-01
   2.83963425e+00  6.93326641e-01 -3.41387424e+00 -2.88472970e+00
   5.29312491e+00  1.80268944e+00]
 [ 6.38032894e-02 -2.06660456e-01  3.41408832e-01  1.22317091e-01
  -6.21068367e-02 -2.33587384e-01  4.12909846e-01  6.99551383e-01
   3.328

iteration 69 x = [[ 1.02720924e-01 -2.38471101e-01  4.17880229e-01 -2.39024373e-02
   1.15059461e-01  9.13275568e-04  5.01485173e-01  5.73406787e-01
   4.01386955e-01  3.28178757e-01  3.03907899e-01  1.46713252e+00
  -1.35696231e+00 -4.94594439e+00 -8.83697144e+00 -2.00283372e+00
   1.14669644e+00  2.30306881e+00]
 [ 1.05091861e-01 -2.34498939e-01  4.34586400e-01 -5.44165034e-02
   6.20934070e-02 -6.33156567e-02  4.87838704e-01  5.64047498e-01
   4.33168951e-01  2.37093691e-01  3.97216195e-01  1.67061704e+00
  -3.05140660e+00 -5.29660544e+00 -6.42289323e+00 -1.36464698e+00
  -9.35928883e-01  3.17819966e+00]], u = [[-0.73918878 -0.13301272  0.01201697 -0.12107553 -0.2266162   0.26333528]]
iteration 70 x = [[ 0.10509186 -0.23449894  0.4345864  -0.0544165   0.06209341 -0.06331566
   0.4878387   0.5640475   0.43316895  0.23709369  0.3972162   1.67061704
  -3.0514066  -5.29660544 -6.42289323 -1.36464698 -0.93592888  3.17819966]
 [ 0.1066792  -0.22900663  0.44953278 -0.09724219  0.02096889 -

...

iteration 917 x = [[ 2.32070425 -0.07040751  1.1755957  -0.143695    0.03208473  0.0147431
   0.03537583  0.41326371  0.12989024  0.28425631 -0.37498036 -0.51288131
  -0.62140659 -0.9878515   0.70619178 -2.08871553 -0.01832822  5.73912842]
 [ 2.3235548  -0.07426031  1.17207135 -0.15219509  0.03465232  0.04013414
   0.03952446  0.40563555  0.20313169  0.28505499 -0.38527969 -0.3524344
  -0.85000931  0.256759    2.53910432  0.41486276 -0.76281643  7.32414435]], u = [[-0.04348927  0.60257574  0.55579664 -2.28783531 -1.39120471 -0.60359875]]
iteration 918 x = [[ 2.32355480e+00 -7.42603065e-02  1.17207135e+00 -1.52195093e-01
   3.46523228e-02  4.01341427e-02  3.95244565e-02  4.05635545e-01
   2.03131687e-01  2.85054986e-01 -3.85279689e-01 -3.52434397e-01
  -8.50009310e-01  2.56758998e-01  2.53910432e+00  4.14862761e-01
  -7.62816433e-01  7.32414435e+00]
 [ 2.32724759e+00 -7.71163512e-02  1.16649147e+00 -1.36408551e-01
  -7.06790580e-03  9.22870608e-02  7.12443293e-02  3.62051913e-01
   2.62

iteration 934 x = [[ 2.35039109 -0.16401328  0.96703685  0.08608783 -0.05932379 -0.08762004
   0.32997523  0.30882612  0.36128372  0.2284254  -1.00731215 -1.93207902
   4.43560656  6.00162668 -3.37979415  0.64089948  3.17996628  2.04783013]
 [ 2.35091347 -0.1750132   0.95021033  0.10972144  0.0123428  -0.07912979
   0.36312783  0.30709799  0.35291229  0.05223787 -1.09999169 -1.68265179
   2.36336104  7.16665882  0.84902539  3.31525967 -0.17281244 -0.83714254]], u = [[-0.31092581  0.29599354  0.54521657  0.91711564 -0.18498414 -0.64634414]]
iteration 935 x = [[ 2.35091347 -0.1750132   0.95021033  0.10972144  0.0123428  -0.07912979
   0.36312783  0.30709799  0.35291229  0.05223787 -1.09999169 -1.68265179
   2.36336104  7.16665882  0.84902539  3.31525967 -0.17281244 -0.83714254]
 [ 2.34922496 -0.18574532  0.93538031  0.09679875  0.10286466 -0.02282797
   0.40774521  0.28918976  0.3257252  -0.16885099 -1.07321239 -1.48300147
  -1.29226941  9.05218562  5.63018204  4.46173839 -1.79082378 -2.

iteration 949 x = [[ 2.37080816 -0.18393013  0.95466791 -0.1025494  -0.04507232  0.02099675
   0.32591095  0.36547802  0.39081646  0.52209176  0.24615311  0.10713448
   1.57433221 -3.49298379  2.03078748 -3.05797021  6.00878627  1.47280958]
 [ 2.37378561 -0.18098139  0.95328923 -0.11254912 -0.03589044  0.03370751
   0.2967333   0.40641357  0.40318988  0.2977458   0.2948735  -0.13786832
  -0.9999717   0.91818782  1.27107563 -2.91776475  4.0935547   1.23734168]], u = [[-0.6585996   0.71452966 -0.0723993  -0.50528648 -0.34749611 -0.04659644]]
iteration 950 x = [[ 2.37378561 -0.18098139  0.95328923 -0.11254912 -0.03589044  0.03370751
   0.2967333   0.40641357  0.40318988  0.2977458   0.2948735  -0.13786832
  -0.9999717   0.91818782  1.27107563 -2.91776475  4.0935547   1.23734168]
 [ 2.37753875 -0.17628609  0.9557894  -0.12499562 -0.04413405  0.07620534
   0.26318255  0.44025158  0.43174403  0.37531351  0.46953005  0.25001697
  -1.24465012 -0.82436118  4.24978335 -3.35507484  3.38380145  2.

iteration 971 x = [[ 2.43431958 -0.12174083  1.06740427 -0.01855957 -0.10272595 -0.11793658
   0.23683553  0.32748055  0.17564165  0.56880192  0.93435137  1.44319421
  -0.02504487 -5.71767293  1.39705151 -3.39457441  1.06936508  2.12017846]
 [ 2.43927169 -0.11232731  1.08052457 -0.02087598 -0.13035208 -0.10896724
   0.20658913  0.32645578  0.20329455  0.49521154  0.94135159  1.31202955
  -0.23164077 -2.76261362  0.89693383 -3.02464015 -0.10247734  2.76528917]], u = [[ 0.3913865   0.50162706 -0.21245774 -1.13887866 -1.10588444 -0.55011461]]
iteration 972 x = [[ 2.43927169 -0.11232731  1.08052457 -0.02087598 -0.13035208 -0.10896724
   0.20658913  0.32645578  0.20329455  0.49521154  0.94135159  1.31202955
  -0.23164077 -2.76261362  0.89693383 -3.02464015 -0.10247734  2.76528917]
 [ 2.44413154 -0.10538378  1.09332578 -0.01560296 -0.14774659 -0.07033867
   0.19629348  0.3031678   0.22529445  0.48598458  0.6943531   1.28012067
   0.52730246 -1.73945061  3.86285675 -1.02956449 -2.32879718  2.

iteration 988 x = [[ 2.51902807 -0.04353309  1.24262858  0.05217161 -0.07982614  0.12946393
   0.07919006  0.22283662  0.08916017  0.47590771  0.31791043  0.8093972
   0.360754    0.46838246  5.14219013  0.82290664 -0.39749212 -4.58750126]
 [ 2.52231105 -0.04012087  1.24696019  0.03772047 -0.03820261  0.16035981
   0.07360649  0.24234136  0.06239478  0.32829811  0.34122161  0.43316141
  -1.44511429  4.16235249  3.08958798 -0.55835771  1.95047338 -2.67653851]], u = [[-0.32598139  0.59327575  0.00724323 -0.09690774  1.26202316  0.73771631]]
iteration 989 x = [[ 2.52231105e+00 -4.01208742e-02  1.24696019e+00  3.77204676e-02
  -3.82026128e-02  1.60359805e-01  7.36064851e-02  2.42341358e-01
   6.23947824e-02  3.28298114e-01  3.41221610e-01  4.33161410e-01
  -1.44511429e+00  4.16235249e+00  3.08958798e+00 -5.58357713e-01
   1.95047338e+00 -2.67653851e+00]
 [ 2.52579105e+00 -3.69907328e-02  1.25158428e+00  2.09152627e-02
  -4.21948061e-04  2.02466615e-01  5.79984332e-02  2.72254666e-01
   4.5

# Rollouts

In [12]:
multi_evaluate(env, dynamics, x0s, us, render=True)

performance
receding_performance


,rewards,lengths
performance,3.258835,15
receding_performance,73.600421,1000
